In [ ]:
import pandas as pd; import numpy as np; import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None # default warn => SettingWithCopyWarning
from collections import namedtuple
from pprint import (pprint as pp, pformat as pf)
import time
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

import dmyplant2
dmyplant2.cred()
mp = dmyplant2.MyPlant(3600)

from dmyplant2 import FSMOperator

def sfun(x):
    return all([
            ("Forsa Hartmoor" in str(x['IB Site Name'])), 
            (x['OperationalCondition'] != 'Decommissioned')
        ])

fleet = mp.search_installed_fleet(sfun).drop('index', axis=1)
fleet = fleet.sort_values(by = "Engine ID",ascending=True).reset_index(drop='index')
fleet.T;
ddl = pd.DataFrame(fleet['serialNumber'] + ' - ' + fleet['IB Site Name'] + ' ' + fleet['Engine ID'])[0].to_list()
ddl = [m for m in ddl]

In [ ]:
import panel as pn
import panel.widgets as pnw
from bokeh.plotting import figure
from panel.layout.gridstack import GridStack, GridSpec
pn.extension()

def sinus4(name='sin', amp=1.0,freq=1.0):
    fig = figure(title=name, height=600, sizing_mode='stretch_width',margin = [10,50,10,10])
    x = np.linspace(-np.pi, np.pi,1000)
    y = np.sin(1/amp*freq*x) * np.sin(freq * x)
    fig.line(x,y)
    return fig

name  = pnw.Select(name='name', value='sin', options=ddl)
amp  = pnw.FloatSlider(name='amp', value=1.0, start=0.0, end=5.0)
freq  = pnw.FloatSlider(name='freq', value=1.0, start=0.0, end=50.0)
modes = pn.widgets.CheckBoxGroup(name='Checkbox Group', value=['Apple', 'Pear'], options=['Apple', 'Banana', 'Pear', 'Strawberry'],inline=True)

reactive_sinus = pn.bind(sinus4, name, amp, freq)
widget1 = pn.Column(
        pn.Row(
                name, 
                modes
            ),
        height=80, 
        margin = [0,0,0,0]
    )
widget2 = pn.Column(f"#Function", amp, freq, width=300, margin = [5,0,0,0])

gspec = pn.GridSpec(height=700, sizing_mode='stretch_width')
gspec[0,0:8] = widget1
gspec[1:12,0] = widget2
gspec[1:12,1:8] = reactive_sinus

fleetwidget = pn.widgets.Tabulator(fleet.T, layout='fit_columns', selectable=False)

tabs = pn.Tabs(('Starts', gspec), ('Fleet',fleetwidget))
tabs